In [1]:
import ijson
import polars as pl
import json

In [2]:
with open("/Users/mz/Downloads/133957095_NewYork-PresbyterianHospital_standardcharges 2.json" , 'r') as file:
    df = json.load(file)



In [9]:
cl = pl.from_dicts(df['standard_charge_information'])


In [11]:
cl = (
    cl
    # 1: explode standard_charges (list → rows)
    .explode("standard_charges")
    .unnest("standard_charges")

    # 2: explode payers_information (list → rows)
    .explode("payers_information")
    .unnest("payers_information")

    # 3: code_information is a list of 1 element → flatten
    .explode("code_information")
    .unnest("code_information")
)

In [16]:
cl.head()

description,drug_information,code,type,setting,gross,discounted_cash,minimum,maximum,payer_name,plan_name,standard_charge_dollar,methodology,additional_payer_notes,modifier_information
str,struct[2],str,str,str,f64,f64,f64,f64,str,str,f64,str,str,struct[1]
"""BABYBIG 100 MG IV SOLR""","{""100"",""ME""}","""90288""","""CPT""","""both""",200550.0,200550.0,48521.87,200550.0,"""1199SEIU Funds""","""All Commercial Plans""",100275.0,"""percent of total billed charge…","""Included in case rate when pro…",null
"""BABYBIG 100 MG IV SOLR""","{""100"",""ME""}","""90288""","""CPT""","""both""",200550.0,200550.0,48521.87,200550.0,"""Aetna""","""Commercial""",63225.19,"""percent of total billed charge…","""Included in case rate when pro…",null
"""BABYBIG 100 MG IV SOLR""","{""100"",""ME""}","""90288""","""CPT""","""both""",200550.0,200550.0,48521.87,200550.0,"""Anthem Blue Cross and Blue Shi…","""All Commercial Plans""",100275.0,"""percent of total billed charge…","""Included in case rate when pro…",null
"""BABYBIG 100 MG IV SOLR""","{""100"",""ME""}","""90288""","""CPT""","""both""",200550.0,200550.0,48521.87,200550.0,"""Cigna Healthcare""","""All Commercial Plans""",63103.06,"""percent of total billed charge…",null,null
"""BABYBIG 100 MG IV SOLR""","{""100"",""ME""}","""90288""","""CPT""","""both""",200550.0,200550.0,48521.87,200550.0,"""EmblemHealth""","""All Commercial Plans""",55937.31,"""fee schedule""","""Included in case rate when pro…",null


In [18]:
cpt = cl.filter(pl.col('code') == '92250')

In [20]:
cpt.columns

['description',
 'drug_information',
 'code',
 'type',
 'setting',
 'gross',
 'discounted_cash',
 'minimum',
 'maximum',
 'payer_name',
 'plan_name',
 'standard_charge_dollar',
 'methodology',
 'additional_payer_notes',
 'modifier_information']

In [28]:
(cpt[['setting', 'payer_name', 'plan_name', 'standard_charge_dollar', 'methodology']]).write_csv("nyp_92250_cpt.csv")